# Design and generate beam codes

## Initialization

In [1]:
import pandas as pd
import numpy as np
import math
import json
import os
from lib import csiapi, util, beam
# from lib import csiapi
# import lib.util as util
import configparser

%load_ext autoreload
%autoreload 2

# Connect to ETABS
# Initialization of ETABSObject
ETABSObject, SapModel = csiapi.get_active_etabs_object()

Succesfully created helper object
Succesfully created SapModel


Initialize settings and default values

In [2]:
SapModel.SetPresentUnits_2(4,6,2) # to kN, m, C

# read config files
config = configparser.ConfigParser()
config.read('config.ini')
concrete_cover = float(config['Concrete']['concrete_cover'])
distance_between_horizontal_reinforcement = float(config['Concrete']['distance_between_horizontal_reinforcement'])
distance_between_vertical_reinforcement = float(config['Concrete']['distance_between_vertical_reinforcement'])
fy = float(config['Steel']['fy'])
dt_assume = float(config['Steel']['dt_assume'])

In [3]:
SapModel.Analyze.RunAnalysis()
SapModel.DesignConcrete.StartDesign()

0

## Create groups in ETABS based on reinforcement. Optional if created manually.
cantilever_prefix must be assigned.

renew_groups can be set to False if manually generated.

In [4]:
# create groups in ETABS based on reinforcement
# Take note of renew_groups

cantilever_prefix = 'C' # MANDATORY
beam_prefix = 'B'
girder_prefix = 'G'
girder_reinforcement = 22
cantilever_reinforcement = 19
beam_reinforcement = 19
renew_groups = True # default is False, use True if script is ran for the first time
smf_flag = True # apply rho = 1/2 and rho = 1/4 manually

# create ETABS groups for each beam diameter
cantilever_reinforcement_group = "D"+str(cantilever_reinforcement)+"_Design"
girder_reinforcement_group = "D"+str(girder_reinforcement)+"_Design"
beam_reinforcement_group = "D"+str(beam_reinforcement)+"_Design"

beam_assignments = csiapi.get_table_display(SapModel, 'Frame Assignments - Summary')

# delete previous design groups if renew_groups is True
if renew_groups:
    group_name_list = SapModel.GroupDef.GetNameList()
    for i in group_name_list[1]:
        if '_Design' in i:
            SapModel.GroupDef.Delete(i)

SapModel.GroupDef.SetGroup_1(cantilever_reinforcement_group)
SapModel.GroupDef.SetGroup_1(girder_reinforcement_group)
SapModel.GroupDef.SetGroup_1(beam_reinforcement_group)

# assign beams according to prefix to each group
if renew_groups:
    for j in range(len(beam_assignments)):
        if cantilever_prefix in beam_assignments['Design Section'][j]:
            SapModel.FrameObj.SetGroupAssign(str(beam_assignments['UniqueName'][j]), cantilever_reinforcement_group, False)
        elif girder_prefix in beam_assignments['Design Section'][j]:
            SapModel.FrameObj.SetGroupAssign(str(beam_assignments['UniqueName'][j]), girder_reinforcement_group, False)
        elif beam_prefix in beam_assignments['Design Section'][j]:
            SapModel.FrameObj.SetGroupAssign(str(beam_assignments['UniqueName'][j]), beam_reinforcement_group, False)

## Assign locations to beams
Each beam station is given a location ('End-I', 'Mid', 'End-J'). Cantilevers are given ('End' and 'Mid')

In [5]:
# Assinging locations to beams
concrete_design_code = SapModel.DesignConcrete.GetCode()
beam_reinforcement_df = csiapi.get_table_display(SapModel, 'Concrete Beam Design Summary - ' + concrete_design_code[0])

In [6]:
beam_reinforcement_df.head()

,Story,Label,UniqueName,DesignSect,Station,Status,AsTopCombo,AsMinTop,AsTop,AsBotCombo,...,VRebar,TLngCombo,TLngRebar,TTrnCombo,TTrnRebar,AsTopTotal,AsBotTotal,AtTrnTotal,WarnMsg,ErrMsg
0,24.0,B148,363,G35,0.15000,No Message,4,0.000433,0.000675,6,...,0.00098,6,0.0,6,0.0,0.000675,0.000433,0.00098,No Message,No Message
1,24.0,B148,363,G35,0.60714,No Message,6,0.000433,0.000433,6,...,0.00084,6,0.0,6,0.0,0.000433,0.000433,0.00084,No Message,No Message
2,24.0,B148,363,G35,1.06429,No Message,6,0.000433,0.000433,4,...,0.00070,6,0.0,6,0.0,0.000433,0.000553,0.00070,No Message,No Message
3,24.0,B148,363,G35,1.52143,No Message,6,0.000433,0.000433,4,...,0.00094,6,0.0,6,0.0,0.000433,0.000651,0.00094,No Message,No Message
4,24.0,B148,363,G35,1.97857,No Message,6,0.000433,0.000433,4,...,0.00069,6,0.0,6,0.0,0.000433,0.000656,0.00069,No Message,No Message


In [7]:
beam.assign_beam_stations(beam_reinforcement_df).head()

,Story,Label,UniqueName,DesignSect,Station,Status,AsTopCombo,AsMinTop,AsTop,AsBotCombo,...,TLngCombo,TLngRebar,TTrnCombo,TTrnRebar,AsTopTotal,AsBotTotal,AtTrnTotal,WarnMsg,ErrMsg,Location
0,24.0,B148,363,G35,0.15000,No Message,4,0.000433,0.000675,6,...,6,0.0,6,0.0,0.000675,0.000433,0.00098,No Message,No Message,End-I
1,24.0,B148,363,G35,0.60714,No Message,6,0.000433,0.000433,6,...,6,0.0,6,0.0,0.000433,0.000433,0.00084,No Message,No Message,End-I
2,24.0,B148,363,G35,1.06429,No Message,6,0.000433,0.000433,4,...,6,0.0,6,0.0,0.000433,0.000553,0.00070,No Message,No Message,Mid
3,24.0,B148,363,G35,1.52143,No Message,6,0.000433,0.000433,4,...,6,0.0,6,0.0,0.000433,0.000651,0.00094,No Message,No Message,Mid
4,24.0,B148,363,G35,1.97857,No Message,6,0.000433,0.000433,4,...,6,0.0,6,0.0,0.000433,0.000656,0.00069,No Message,No Message,Mid


## Data processing & Asmin calculation

In [8]:
# Joining tables
group_assignments_df = csiapi.get_table_display(SapModel, 'Group Assignments')
group_assignments_df = group_assignments_df.drop(columns = ['Object Type'])[group_assignments_df['Group Name'].str.contains('_Design')]

In [9]:
# create a table with only unique names of beam design results
beam_unique_name_df = pd.DataFrame(beam_reinforcement_df['UniqueName'].unique(), columns = ['UniqueName'])

beam_assignments = csiapi.get_table_display(SapModel, 'Frame Assignments - Summary')
beam_rectangular_sections = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Rectangular')
concrete_materials = csiapi.get_table_display(SapModel, 'Material Properties - Concrete Data')

beam_assignments = beam_assignments[['UniqueName', 'Design Section']]
beam_rectangular_sections = beam_rectangular_sections[['Name', 'Material', 'Depth', 'Width']]
concrete_materials = concrete_materials[['Material', 'Fc']] # Fc' in kN/m^2

beam_rectangular_sections = pd.merge(beam_rectangular_sections, concrete_materials, how = 'inner')
beam_unique_name_df = pd.merge(beam_unique_name_df, beam_assignments, how = 'inner', on = 'UniqueName')
beam_unique_name_df = pd.merge(beam_unique_name_df, beam_rectangular_sections, how = 'inner', left_on = 'Design Section', right_on = 'Name', copy = False)
beam_unique_name_df = pd.merge(beam_unique_name_df, group_assignments_df, how = 'inner', left_on = 'UniqueName', right_on = 'Object Unique Name')
beam_unique_name_df.drop(columns = ['Name', 'Object Unique Name'], inplace = True)
# simplify both end-I and end-J to end
as_top_end = []
as_bot_end = []
as_top_mid = []
as_bot_mid = []
as_min_flag = []
# assume cover of 60 mm
rebar_size = ((beam_unique_name_df['Group Name'].str.extract('(\d+)')).astype(int))
effective_area = (beam_unique_name_df['Depth'] - concrete_cover - dt_assume / 1000 - rebar_size.iloc[:,0] / 2000) * beam_unique_name_df['Width']

# calculate Asmin
as_min_coefficient = pd.DataFrame(0.25*(beam_unique_name_df['Fc']/1000)**0.5)
as_min_coefficient['coeff2'] = 1.4
as_min_coefficient = as_min_coefficient.max(axis=1) / fy # might need to reconfigure
as_min = (as_min_coefficient * effective_area)

for i in beam_unique_name_df['UniqueName']:
    local_beam_table = beam_reinforcement_df[i == beam_reinforcement_df['UniqueName']]
    local_as_top_end = (local_beam_table['AsTopTotal'][local_beam_table['Location'].str.contains('End')]).max()
    local_as_bot_end = (local_beam_table['AsBotTotal'][local_beam_table['Location'].str.contains('End')]).max()
    local_as_top_mid = (local_beam_table['AsTopTotal'][local_beam_table['Location'].str.contains('Mid')]).max()
    local_as_bot_mid = (local_beam_table['AsBotTotal'][local_beam_table['Location'].str.contains('Mid')]).max()
    as_top_end.append(local_as_top_end)
    as_bot_end.append(local_as_bot_end)
    as_top_mid.append(local_as_top_mid)
    as_bot_mid.append(local_as_bot_mid)
    as_min_flag.append(all((local_beam_table['AsMinTop'] == local_beam_table['AsTopTotal']) & (local_beam_table['AsMinBot'] == local_beam_table['AsBotTotal'])))
    pass

In [10]:
as_top_end = pd.Series(as_top_end)
as_bot_end = pd.Series(as_bot_end)
as_top_mid = pd.Series(as_top_mid)
as_bot_mid = pd.Series(as_bot_mid)
beam_unique_name_df['AsTopEnd'] = as_top_end
beam_unique_name_df['AsBotEnd'] = as_bot_end
beam_unique_name_df['AsTopMid'] = as_top_mid
beam_unique_name_df['AsBotMid'] = as_bot_mid
beam_unique_name_df['AsMin'] = as_min

beam_unique_name_df_1 = beam_unique_name_df.copy(deep = True)

as_top_end[as_top_end < as_min] = as_min[as_top_end < as_min]
as_bot_end[as_bot_end < as_min] = as_min[as_bot_end < as_min]
as_top_mid[as_top_mid < as_min] = as_min[as_top_mid < as_min]
as_bot_mid[as_bot_mid < as_min] = as_min[as_bot_mid < as_min]
# beam_unique_name_df = beam_unique_name_df
beam_unique_name_df['AsTopEnd'] = as_top_end
beam_unique_name_df['AsBotEnd'] = as_bot_end
beam_unique_name_df['AsTopMid'] = as_top_mid
beam_unique_name_df['AsBotMid'] = as_bot_mid
beam_unique_name_df['AsMin'] = as_min

beam_all_as_min = as_min > (beam_unique_name_df_1[['AsTopEnd', 'AsBotEnd', 'AsTopMid', 'AsBotMid']]).max(axis = 1)
beam_as_min_flag = pd.concat([beam_all_as_min, pd.Series(as_min_flag)], axis=1).any(axis=1)

beam_all_as_min_table = beam_unique_name_df_1[beam_as_min_flag]

## Checkpoint 1: Beams with Asmin included

In [11]:
beam_all_as_min_table

,UniqueName,Design Section,Material,Depth,Width,Fc,Group Name,AsTopEnd,AsBotEnd,AsTopMid,AsBotMid,AsMin
27,193,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000433,0.000433,0.000433,0.000433,0.000439
28,79,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000433,0.000433,0.000433,0.000433,0.000439
51,190,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339
52,196,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339
53,199,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339
54,201,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339
55,203,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339
56,230,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339
112,184,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339
113,191,G34,25 MPA,0.4,0.30,25000,D22_Design,0.000335,0.000335,0.000335,0.000335,0.000339


In [12]:
display(beam_unique_name_df)

,UniqueName,Design Section,Material,Depth,Width,Fc,Group Name,AsTopEnd,AsBotEnd,AsTopMid,AsBotMid,AsMin
0,363,G35,25 MPA,0.5,0.3,25000,D22_Design,0.000702,0.000439,0.000439,0.000656,0.000439
1,665,G35,25 MPA,0.5,0.3,25000,D22_Design,0.000702,0.000439,0.000439,0.000656,0.000439
2,242,G35,25 MPA,0.5,0.3,25000,D22_Design,0.000653,0.000453,0.000439,0.000554,0.000439
3,244,G35,25 MPA,0.5,0.3,25000,D22_Design,0.000653,0.000453,0.000439,0.000554,0.000439
4,258,G35,25 MPA,0.5,0.3,25000,D22_Design,0.000659,0.000452,0.000439,0.000550,0.000439
...,...,...,...,...,...,...,...,...,...,...,...,...
384,305,G46,25 MPA,0.6,0.4,25000,D22_Design,0.001967,0.001678,0.000934,0.000870,0.000719
385,155,G46,25 MPA,0.6,0.4,25000,D22_Design,0.003212,0.001796,0.001765,0.000719,0.000719
386,506,G46,25 MPA,0.6,0.4,25000,D22_Design,0.003215,0.001797,0.001767,0.000719,0.000719
387,195,B46,25 MPA,0.6,0.4,25000,D19_Design,0.001126,0.000938,0.000721,0.001557,0.000721


## Calculate number of longitudinal reinforcement required
Results without Asmin is saved as beam_unique_name_df_1

In [13]:
# Get Asbar for each beam
rebar_area = (rebar_size/1000) ** 2 * math.pi / 4

no_rebar_required = beam_unique_name_df[['AsTopEnd','AsBotEnd','AsTopMid','AsBotMid']]/rebar_area.to_numpy()
no_rebar_required = no_rebar_required.rename(columns={'AsTopEnd': 'RebarTopEnd',
                                                      'AsBotEnd': 'RebarBotEnd',
                                                      'AsTopMid': 'RebarTopMid',
                                                      'AsBotMid': 'RebarBotMid'})

no_rebar_required_rounded = (no_rebar_required.apply(np.ceil)).astype(int) # convert to int to remove decimals then convert to str

no_rebar_required_rounded.mask(no_rebar_required_rounded.astype(int)<2,2, inplace = True) # minimum of 2 rein

In [14]:
max_per_layers = (beam_unique_name_df['Width'] - (2 * (concrete_cover + dt_assume/1000)) + distance_between_horizontal_reinforcement) / (distance_between_horizontal_reinforcement + rebar_size.loc[:,0]/1000)
max_per_layers = np.floor(max_per_layers).astype(int)

# max_layers = np.floor(no_rebar_required_rounded.div(max_per_layers,axis=0))
number_layers = np.ceil(no_rebar_required_rounded.astype(int).div(max_per_layers, axis = 0))
last_layer_rein = no_rebar_required_rounded.astype(int).mod(max_per_layers, axis = 0)

for i in last_layer_rein.items():
    _, i = i
    i[i == 0] = max_per_layers[i == 0]

# util.open_as_csv(pd.concat([last_layer_rein,number_layers,no_rebar_required_rounded],axis=1))

adjusted_cover_simplified = number_layers.copy(deep = True)

real_cover = 0
for i in range(adjusted_cover_simplified.shape[0]):
    for j in range(adjusted_cover_simplified.shape[1]):
        n_layer = number_layers.iloc[i,j]
        cover_array = []
        current_rebar_size = rebar_size.iloc[i,0]
        for k in range(int(n_layer)):
            current_cover = concrete_cover + dt_assume/1000 + (current_rebar_size/1000 / 2 * (2*k + 1)) + distance_between_vertical_reinforcement * k
            cover_array.append(current_cover)
            # cover_array[k] = concrete_cover + dt_assume/1000 + (rebar_size[i]/1000 / 2 * (2*k - 1)) + distance_between_vertical_reinforcement * k
            
        real_cover = (sum(cover_array[:-1]) * max_per_layers[i] + last_layer_rein.iloc[i,j] * cover_array[-1]) / no_rebar_required_rounded.iloc[i,j]
        adjusted_cover_simplified.iloc[i,j] = real_cover
    #adjusted_cover_simplified.iloc[i,j] = real_cover

In [15]:
# new rho calculations
etabs_default = 0.06
fc = beam_unique_name_df['Fc']/1000
beam_reinforcement_required = beam_unique_name_df[['AsTopEnd','AsBotEnd','AsTopMid','AsBotMid']]
original_d = (beam_unique_name_df['Depth'] - etabs_default)
new_d = -adjusted_cover_simplified.sub(beam_unique_name_df['Depth'],axis=0)
rho_1 = beam_reinforcement_required.div(beam_unique_name_df['Width'] * (beam_unique_name_df['Depth'] - etabs_default),axis=0)
R1 = (1-(0.588 * rho_1 * fy).div(fc,axis=0))*rho_1*fy
R2 = new_d.div(original_d,axis=0).pow(-2)
R2.columns = R1.columns
R2 = R2*R1
rho_2 = -((- fy + (-((2.352 * R2 * fy ** 2).sub(fc * fy **2, axis = 0))).div(fc,axis=0).pow(1/2)).mul(fc,axis=0)) / (1.176 * fy ** 2)
new_d.columns = rho_2.columns
as_corrected = rho_2.mul(new_d).mul(beam_unique_name_df['Width'],axis=0)
beam_unique_name_df_corrected = beam_unique_name_df.copy(deep = True)
beam_unique_name_df_corrected[['AsTopEnd','AsBotEnd','AsTopMid','AsBotMid']] = as_corrected

In [16]:
beam_unique_name_df = beam_unique_name_df_corrected

no_rebar_required = beam_unique_name_df[['AsTopEnd','AsBotEnd','AsTopMid','AsBotMid']]/rebar_area.to_numpy()
no_rebar_required = no_rebar_required.rename(columns={'AsTopEnd': 'RebarTopEnd',
                                                      'AsBotEnd': 'RebarBotEnd',
                                                      'AsTopMid': 'RebarTopMid',
                                                      'AsBotMid': 'RebarBotMid'})

no_rebar_required_rounded = (no_rebar_required.apply(np.ceil)).astype(int) # convert to int to remove decimals then convert to str

no_rebar_required_rounded.mask(no_rebar_required_rounded.astype(int)<2,2, inplace = True) # minimum of 2 rein
# overwrite unique_name_table

In [17]:
df = no_rebar_required_rounded.astype(int)

# df[df.max(axis = 1) < df] = [df.max(axis = 1) < df]

if smf_flag:
    smf_beams = df[beam_unique_name_df['Design Section'].str.contains(girder_prefix)]
    smf_beams.loc[:,('RebarBotEnd')] = np.maximum((smf_beams.loc[:,('RebarTopEnd')]/2).apply(np.ceil),smf_beams.loc[:,('RebarBotEnd')])
    max_series = smf_beams.max(axis=1)
    max_df = max_series.to_frame()
    # Apply np.maximum to two DataFrames
    for column in smf_beams:
        smf_beams.loc[:,column] = np.maximum(np.ceil(max_series / 4), smf_beams.loc[:,column])
    df[beam_unique_name_df['Design Section'].str.contains(girder_prefix)] = smf_beams
    no_rebar_required_rounded.astype(int)

no_rebar_required_label = no_rebar_required_rounded.copy(deep = True)
no_rebar_required_rounded = no_rebar_required_rounded.astype(str)
for column in no_rebar_required_rounded:
    # print(type(column))
    no_rebar_required_label[column] = no_rebar_required_rounded[column].str.cat(rebar_size.astype(str),sep = "D")

beam_unique_name_df.join(no_rebar_required_label)

beam_sections_reinforcement = pd.concat([beam_unique_name_df[['Design Section']], no_rebar_required_label], axis = 1)
beam_sections_reinforcement_numbers = pd.concat([beam_unique_name_df[['Design Section']], no_rebar_required_rounded], axis = 1)
# beam_sections_reinforcement = beam_sections_reinforcement.merge(no_rebar_required_rounded)
beam_sections_reinforcement

unique_beam_sections_reinforcement = (beam_sections_reinforcement_numbers.drop_duplicates())

unique_beam_sections_reinforcement_count = beam_sections_reinforcement.value_counts().reset_index()

unique_beam_sections_reinforcement
# unique_beam_sections_reinforcement_count.reset_index()

,Design Section,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid
0,G35,2,2,2,2
6,G35,3,2,2,3
9,G35,4,2,2,3
14,G35,3,3,2,2
15,G35,3,2,2,2
...,...,...,...,...,...
374,G46,3,5,2,8
381,G46,2,3,7,5
383,G46,6,5,3,3
385,G46,9,5,5,2


## Checkpoint 2: Unique beam reinforcing and the its count

## Grouping parameters

In [18]:
sort_by = [1,4,2,3] # Rebar top end, bot mid, bot end, top mid
max_total_end_difference = 2 # for joints
max_total_difference = 4
upwards_tolerance = 1
round_botmid_to_botend = True

sort_dict = {
    1: 'RebarTopEnd',
    2: 'RebarBotEnd',
    3: 'RebarTopMid',
    4: 'RebarBotMid'}
sort_by_string = []
for i in range(len(sort_by)):
    sort_by_string.append(sort_dict[sort_by[i]])

In [19]:
rebar_size.rename(mapper = {0: 'Rebar size'}, axis = 1, inplace = True)

no_rebar_required_and_size = pd.concat([beam_unique_name_df[['Design Section']], no_rebar_required_rounded, rebar_size], axis = 1)
no_rebar_required_and_size[sort_by_string] = no_rebar_required_and_size[sort_by_string].astype(int)

if round_botmid_to_botend:
    no_rebar_required_and_size['RebarBotMid'] = no_rebar_required_and_size[['RebarBotMid', 'RebarBotEnd']].max(axis=1)

In [20]:
unique_beam_sections_reinforcement[sort_by_string] = unique_beam_sections_reinforcement[sort_by_string].astype(int)
unique_beam_sections_reinforcement_sorted = unique_beam_sections_reinforcement.sort_values(by = sort_by_string, ascending = False)
unique_beam_sections_reinforcement_sorted = pd.concat([unique_beam_sections_reinforcement_sorted, rebar_size], axis = 1, join = 'inner')

pd.set_option('display.max_rows',500)
unique_beam_sections_reinforcement_sorted.columns = unique_beam_sections_reinforcement_sorted.columns.str.replace(' ','_')
display(unique_beam_sections_reinforcement_sorted)
display(unique_beam_sections_reinforcement_sorted.query('Design_Section == "B4070"')) # pairs with no_rebar_required_and_size

C:\Users\nico\AppData\Local\Temp\ipykernel_4896\3410948447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_beam_sections_reinforcement[sort_by_string] = unique_beam_sections_reinforcement[sort_by_string].astype(int)


,Design_Section,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar_size
313,G457,13,6,4,10,22
314,G457,12,6,3,9,22
209,G47,10,7,3,10,22
373,G46,10,5,3,6,22
312,G457,10,6,3,4,22
208,G47,9,7,3,10,22
191,G47,9,6,4,6,22
202,G47,9,7,4,5,22
309,G457,9,6,3,4,22
385,G46,9,5,5,2,22


,Design_Section,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar_size


In [21]:
beam_unique_name_df_2 = pd.concat([beam_unique_name_df, no_rebar_required_and_size], axis = 1)

df_temp = unique_beam_sections_reinforcement_sorted.copy(deep = True)
df_temp[sort_by_string] = df_temp[sort_by_string].astype(int)
df_temp2 = beam_unique_name_df_2.copy(deep = True)
df_temp2[sort_by_string] = df_temp2[sort_by_string].astype(int)
df_temp.reset_index(inplace = True)
df_temp.drop(columns = 'index', inplace = True)
# df_temp2 = df_temp2.loc[:,~df_temp2.columns.duplicated()]

df_temp2 = df_temp2.loc[:,~df_temp2.columns.duplicated()]

#df_temp2.drop(['Material', 'Depth', 'Width', 'Fc', 'Group Name', 'AsTopEnd', 'AsBotEnd', 'AsTopMid', 'AsBotMid', 'AsMin'], axis=1, inplace = True)
df_temp2.drop(['Fc', 'Group Name', 'AsTopEnd', 'AsBotEnd', 'AsTopMid', 'AsBotMid', 'AsMin'], axis=1, inplace = True)

df_temp2.sort_values(by=sort_by_string,inplace = True, ascending = False)

beam_grouping = pd.DataFrame([])
beam_grouping_detail = pd.DataFrame([])
unique_section = df_temp2['Design Section'].unique().tolist()
unique_section_counter = np.zeros(len(unique_section), dtype='int8')
counter = 1
#df_temp2.sort_values(by='RebarTopEnd', ascending = False, inplace=True)
while len(df_temp2) > 0:
    df_temp.reset_index(inplace = True)
    df_temp.drop(columns = 'index', inplace = True)
    df_temp2.reset_index(inplace = True)
    df_temp2.drop(columns = 'index', inplace = True)
    
    current_beam = df_temp2.loc[0,:].transpose().copy()
    current_beam_upwards = current_beam.copy(deep = True)
    current_beam_upwards[sort_by_string] = current_beam_upwards[sort_by_string] + upwards_tolerance

    current_beam_section = current_beam['Design Section']
    unique_section_idx = unique_section.index(current_beam_section)
    unique_section_counter[unique_section_idx] += 1
    current_beam_name = current_beam_section + '-' + str(unique_section_counter[unique_section_idx])

    cond1 = current_beam_upwards['Design Section'] == df_temp2['Design Section']
    cond2 = current_beam_upwards['Rebar size'] == df_temp2['Rebar size']
    cond3 = current_beam_upwards[sort_by_string] >= df_temp2[sort_by_string]
    cond4 = current_beam_upwards[sort_by_string].sum() <= (df_temp2[sort_by_string].sum(axis=1) + max_total_difference + 4*upwards_tolerance)
    cond5 = current_beam_upwards[['RebarTopEnd','RebarBotEnd']].sum() <= (df_temp2[['RebarTopEnd','RebarBotEnd']].sum(axis=1) + max_total_end_difference + 2*upwards_tolerance)
    
    conddf = pd.concat([cond1, cond2, cond3, cond4, cond5],axis=1)
    allcond = conddf.all(axis=1)
    applicable_beams = df_temp2.loc[allcond,:]

    new_current_beam = current_beam.copy(deep = True)
    new_current_beam['RebarTopEnd'] = applicable_beams['RebarTopEnd'].max()
    new_current_beam['RebarBotEnd'] = applicable_beams['RebarBotEnd'].max()
    new_current_beam['RebarTopMid'] = applicable_beams['RebarTopMid'].max()
    new_current_beam['RebarBotMid'] = applicable_beams['RebarBotMid'].max()

    new_current_beam.at['Design Section'] = current_beam_name
    # current_beam['Design Section'].replace(current_beam_section, current_beam_name)
    applicable_beams.loc[:,['New Beam Label']] = current_beam_name
    # print(current_beam)
    
    beam_grouping = pd.concat([beam_grouping, applicable_beams], axis = 0)
    beam_grouping_detail = pd.concat([beam_grouping_detail,new_current_beam],axis=1)
    beam_grouping_detail_1 = beam_grouping_detail.transpose()
    df_temp2 = df_temp2[~allcond]
    # counter += 1
    # if counter > 7:
    #     break
    # df_temp = df_temp.loc[1:,:]

beam_grouping_detail_1.drop(columns = 'UniqueName', inplace = True)
#util.open_as_csv(beam_grouping)
#util.open_as_csv(beam_grouping_detail_1)

beam_grouping_detail_1

,Design Section,Material,Depth,Width,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar size
0,G457-1,25 MPA,0.7,0.45,13,6,4,10,22
0,G47-1,25 MPA,0.7,0.4,10,7,4,10,22
0,G457-2,25 MPA,0.7,0.45,10,6,3,6,22
0,G46-1,25 MPA,0.6,0.4,10,5,3,6,22
0,G47-2,25 MPA,0.7,0.4,9,6,4,6,22
0,G46-2,25 MPA,0.6,0.4,9,5,5,5,22
0,G47-3,25 MPA,0.7,0.4,8,4,3,7,22
0,G457-3,25 MPA,0.7,0.45,8,4,3,4,22
0,G47-4,25 MPA,0.7,0.4,7,6,3,8,22
0,G46-3,25 MPA,0.6,0.4,7,3,2,5,22


In [22]:
new_current_beam

UniqueName           679
Design Section     G45-2
Material          25 MPA
Depth                0.5
Width                0.4
RebarTopEnd            2
RebarBotEnd            2
RebarTopMid            2
RebarBotMid            3
Rebar size            22
Name: 0, dtype: object

## Correction of beam reinforcement

In [23]:
# rho 1/2 and 1/4 correction
cond1 = (beam_grouping_detail_1['RebarTopEnd']/2).ge(beam_grouping_detail_1['RebarBotEnd'])
beam_grouping_detail_1.loc[cond1, 'RebarBotEnd'] = np.ceil(beam_grouping_detail_1.loc[cond1, 'RebarTopEnd']/2)
max_rein_per_row = beam_grouping_detail_1[['RebarTopEnd','RebarBotEnd','RebarTopMid','RebarBotMid']].max(axis=1)
cond2 = (max_rein_per_row/4).ge(beam_grouping_detail_1['RebarTopMid'])

In [24]:
beam_grouping_detail_1.loc[cond2, 'RebarTopMid'] = np.ceil(max_rein_per_row.loc[cond2]/4)

beam_grouping_detail_1.index = np.arange(len(beam_grouping_detail_1))

display(beam_grouping_detail_1)

,Design Section,Material,Depth,Width,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar size
0,G457-1,25 MPA,0.7,0.45,13,7,4,10,22
1,G47-1,25 MPA,0.7,0.4,10,7,4,10,22
2,G457-2,25 MPA,0.7,0.45,10,6,3,6,22
3,G46-1,25 MPA,0.6,0.4,10,5,3,6,22
4,G47-2,25 MPA,0.7,0.4,9,6,4,6,22
5,G46-2,25 MPA,0.6,0.4,9,5,5,5,22
6,G47-3,25 MPA,0.7,0.4,8,4,3,7,22
7,G457-3,25 MPA,0.7,0.45,8,4,3,4,22
8,G47-4,25 MPA,0.7,0.4,7,6,3,8,22
9,G46-3,25 MPA,0.6,0.4,7,4,2,5,22


### Remake beam grouping 

In [25]:
beam_unique_name_df_2 = beam_unique_name_df_2.loc[:,~beam_unique_name_df_2.columns.duplicated()].copy()
beam_sections_and_rebar_size = beam_unique_name_df_2[['Design Section', 'Rebar size','AsMin']].drop_duplicates()
beam_grouping_detail_sizes = beam_grouping_detail_1['Design Section'].str.split('-',expand=True)[0]
new_beam_label = beam_unique_name_df['Design Section'].copy(deep = True).rename({'Design Section':'Beam label'})
beam_grouping_2 = beam_unique_name_df_2.copy()

In [26]:
for idx in range(beam_unique_name_df_2.shape[0]):
    i = beam_unique_name_df_2.iloc[[idx]]
    i[sort_by_string].astype(int)
    # print(i)
    df.index = np.arange(1, len(df)+1)

    
    cond1 = i['Design Section'].iloc[0] == beam_grouping_detail_sizes
    cond2 = i['Rebar size'].iloc[0] == beam_grouping_detail_1['Rebar size']
    cond3 = beam_grouping_detail_1[sort_by_string].ge(i[sort_by_string].astype(int)).all(axis=1)
    cond3 = np.greater_equal(beam_grouping_detail_1[sort_by_string].to_numpy(),i[sort_by_string].astype(int).to_numpy()).all(axis=1)
    cond2.index = cond1.index
    cond3 = pd.DataFrame(cond3)
    #cond3.index = cond1.index
        
    #print(cond1,cond2,cond3)
    suitable_beams_idx = pd.concat([cond1,cond2,cond3],axis=1).all(axis=1)
    suitable_beams = beam_grouping_detail_1.loc[suitable_beams_idx,:]
    subtraction_array = np.subtract((suitable_beams[sort_by_string].astype(int)).to_numpy()
                                    ,i[sort_by_string].astype(int).to_numpy()).sum(axis=1)
    minidx = np.argmin(subtraction_array)
    new_beam = suitable_beams.iloc[minidx]
    new_beam_label.loc[idx] = new_beam['Design Section']
    # print(cond1,cond2,cond3,subtraction_array,minidx)
    #print(cond1,cond2,cond3,cond4)
    pass

beam_grouping_2['New Beam Label'] = new_beam_label

In [27]:
beam_unique_name_df_2

,UniqueName,Design Section,Material,Depth,Width,Fc,Group Name,AsTopEnd,AsBotEnd,AsTopMid,AsBotMid,AsMin,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar size
0,363,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000704,0.000440,0.000440,0.000658,0.000439,2,2,2,2,22
1,665,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000704,0.000440,0.000440,0.000658,0.000439,2,2,2,2,22
2,242,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000655,0.000454,0.000440,0.000555,0.000439,2,2,2,2,22
3,244,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000655,0.000454,0.000440,0.000555,0.000439,2,2,2,2,22
4,258,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000661,0.000453,0.000440,0.000551,0.000439,2,2,2,2,22
5,262,G35,25 MPA,0.5,0.30,25000,D22_Design,0.000661,0.000453,0.000440,0.000551,0.000439,2,2,2,2,22
6,479,G35,25 MPA,0.5,0.30,25000,D22_Design,0.001127,0.000537,0.000440,0.000919,0.000439,3,2,2,3,22
7,504,G35,25 MPA,0.5,0.30,25000,D22_Design,0.001068,0.000574,0.000440,0.001069,0.000439,3,2,2,3,22
8,668,G35,25 MPA,0.5,0.30,25000,D22_Design,0.001068,0.000574,0.000440,0.001069,0.000439,3,2,2,3,22
9,677,G35,25 MPA,0.5,0.30,25000,D22_Design,0.001248,0.000731,0.000440,0.001095,0.000439,4,2,2,3,22


## beam_grouping_detail_1 and beam_grouping

### Create new sections in ETABS

In [28]:
SapModel.SetModelIsLocked(False)

concrete_rectangular_table = csiapi.database_tables(SapModel, 'Frame Section Property Definitions - Concrete Rectangular')
concrete_reinforcement_table = csiapi.database_tables(SapModel, 'Frame Section Property Definitions - Concrete Beam Reinforcing')

concrete_rectangular_table.get_table_edit()
concrete_reinforcement_table.get_table_edit()

# util.open_as_csv(concrete_rectangular_table.table)

label_exists = beam_grouping_detail_1['Design Section'].isin(concrete_rectangular_table.table['Name'])
overwrite_beams = beam_grouping_detail_1.loc[label_exists]
new_beams = beam_grouping_detail_1.loc[~label_exists]

def extract_base(data_string):
    parts = data_string.split('-')
    base_string = '-'.join(parts[:-1])
    return base_string

for i in new_beams.iterrows():
    _, i = i
    beam_section = extract_base(i['Design Section'])
    old_beam_section_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == beam_section, :]
    new_beam = old_beam_section_row
    new_beam['Name'] = i['Design Section']
    new_beam['Color'] = ''
    new_beam['GUID'] = ''
    new_beam['Notes'] = ''
    concrete_rectangular_table.table = pd.concat([concrete_rectangular_table.table, new_beam], ignore_index = True, axis = 0)
    
    as_end_top = i['RebarTopEnd']*math.pi*pow(i['Rebar size']/1000,2)/4
    as_end_bot = i['RebarBotEnd']*math.pi*pow(i['Rebar size']/1000,2)/4
    old_beam_section_reinforcement = concrete_reinforcement_table.table.loc[concrete_reinforcement_table.table['Name'] == beam_section, :]
    new_beam_reinforcement = old_beam_section_reinforcement
    new_beam_reinforcement['Name'] = i['Design Section']
    new_beam_reinforcement['Top I-End Area'] = as_end_top*1000
    new_beam_reinforcement['Top J-End Area'] = as_end_top*1000
    new_beam_reinforcement['Bottom I-End Area'] = as_end_bot*1000
    new_beam_reinforcement['Bottom J-End Area'] = as_end_bot*1000
    concrete_reinforcement_table.table = pd.concat([concrete_reinforcement_table.table, new_beam_reinforcement], ignore_index = True, axis = 0)

for i in overwrite_beams.iterrows():
    _,i = i
    beam_section = extract_base(i['Design Section'])
    old_beam_coded_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == i['Design Section'], :]
    old_beam_section_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == beam_section, :]
    # print(old_beam_section_row)
    new_row = old_beam_section_row
    new_row['Name'] = i['Design Section']
    new_row['Color'] = ''
    new_row['GUID'] = ''
    new_row['Notes'] = ''
    new_row.rename(index = {new_row.index.values[0]: old_beam_coded_row.index.values[0]}, inplace = True)
    concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == i['Design Section'],:] = new_row

    as_end_top = i['RebarTopEnd']*math.pi*pow(i['Rebar size']/1000,2)/4
    as_end_bot = i['RebarBotEnd']*math.pi*pow(i['Rebar size']/1000,2)/4

    old_beam_section_reinforcement = concrete_reinforcement_table.table.loc[concrete_reinforcement_table.table['Name'] == beam_section, :]
    new_reinforcement_row = old_beam_section_reinforcement
    new_reinforcement_row['Name'] = i['Design Section']
    new_reinforcement_row['Top I-End Area'] = as_end_top
    new_reinforcement_row['Top J-End Area'] = as_end_top
    new_reinforcement_row['Bottom I-End Area'] = as_end_bot
    new_reinforcement_row['Bottom J-End Area'] = as_end_bot
    concrete_reinforcement_table.table.loc[concrete_reinforcement_table.table['Name'] == i['Design Section'],:] = new_reinforcement_row

old_beam_section_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == beam_section, :]

concrete_rectangular_table.set_table_edit()
concrete_reinforcement_table.set_table_edit()
SapModel.DatabaseTables.ApplyEditedTables(True)

C:\Users\nico\AppData\Local\Temp\ipykernel_4896\965447932.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_beam['Name'] = i['Design Section']
C:\Users\nico\AppData\Local\Temp\ipykernel_4896\965447932.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_beam['Color'] = ''
C:\Users\nico\AppData\Local\Temp\ipykernel_4896\965447932.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

[0,
 0,
 0,
 0,
 '********** DATABASE IMPORT DEFINITION DATA **********\r\n\r\nETABS Version 21.0.1 Interactive Database Import\r\nImport Type and Options:  Interactive Database Editing\r\n     Elements Deleted from DB:  Delete object from model\r\n     Other Items Deleted from DB:  Delete item from model\r\n     Elements with Same Name in Model and DB:  Replace object in model\r\n     Other Items with Same Name in Model and DB:  Replace item in model\r\n     Elements (Except Links) in the Same Location:  Allow duplicates in the same location\r\n     Link Elements in the Same Location:  Allow duplicates in the same location\r\n     Items With Same Name in a DB Table:  Overwrite previous item\r\n     Import Control for Errors:  Abort import after 20 errors\r\n     Import Control for Warnings:  Abort import after 200 warnings\r\n\r\nImport started at \r\n\r\nImport completed at 15:04:42\r\nNumber of errors:  0\r\nNumber of warnings:  0\r\nNumber of informational messages:  0\r\n\r\n\r\n*

## Assign to beams

In [29]:
beam_grouping

,UniqueName,Design Section,Material,Depth,Width,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar size,New Beam Label
0,59,G457,25 MPA,0.7,0.45,13,6,4,10,22,G457-1
1,517,G457,25 MPA,0.7,0.45,13,6,4,10,22,G457-1
2,66,G457,25 MPA,0.7,0.45,12,6,3,9,22,G457-1
3,518,G457,25 MPA,0.7,0.45,12,6,3,9,22,G457-1
0,224,G47,25 MPA,0.7,0.40,10,7,3,10,22,G47-1
1,570,G47,25 MPA,0.7,0.40,10,7,3,10,22,G47-1
5,220,G47,25 MPA,0.7,0.40,9,7,3,10,22,G47-1
6,566,G47,25 MPA,0.7,0.40,9,7,3,10,22,G47-1
7,326,G47,25 MPA,0.7,0.40,9,7,4,7,22,G47-1
8,632,G47,25 MPA,0.7,0.40,9,7,4,7,22,G47-1


In [30]:
for i in beam_grouping_2.iterrows():
    _, i = i
    SapModel.FrameObj.SetSection(str(i['UniqueName']), str(i['New Beam Label']))

In [31]:
beam_grouping_2.sort_values('New Beam Label', inplace = True)
beam_grouping_detail_1.sort_values('Design Section', inplace = True)

util.open_as_excel(beam_grouping_2,beam_grouping_detail_1)